# NLP Assignment #2
### by Prodromos Kampouridis 

### Γ. Text Classification with RNNs

#### IMPORTANT NOTE
##### *Due to the large length of code, the answers to task Γ can also be found as markdowns in the cells below.*


##### *For more detailed information, please refer to the report entitled PRODROMOS KAMPOURIDIS REPORT*

Firstly, we modify the given code in order to create some reusable functions for each subquestion.The following cell containes the modified data preprocessing, model training and model evaluation code. Furthermore, we move the functionality of data preprocessing under a function named "preprocess_data" that receives the max_words variable as a parameter, so that we can later reuse the same code to produce data loaders and vocabulary based for different max_words value for task 3. Moreover, we alter the TrainModel function to calculate the average epoch time cost in seconds and return it.

In [ ]:
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""
import time
import random

import torch
import numpy as np

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

######################################################################
# Data processing 
# -----------------------------


tokenizer = get_tokenizer("basic_english")

def preprocess_data(max_words, vocab=None):

    # All texts are truncated and padded to max_words tokens
    def collate_batch(batch):
        Y, X = list(zip(*batch))
        Y = torch.tensor(Y) - 1 # Target names in range [0,1,2,3] instead of [1,2,3,4]
        X = [vocab(tokenizer(text)) for text in X]
        # Bringing all samples to max_words length. Shorter texts are padded with <PAD> sequences, longer texts are truncated.
        X = [tokens+([vocab['<PAD>']]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X]
        return torch.tensor(X, dtype=torch.int32).to(device), Y.to(device) 

    train_dataset = [(label,train_data['Title'][i] + ' ' + train_data['Description'][i]) for i,label in enumerate(train_data['Class Index'])]
    test_dataset = [(label,test_data['Title'][i] + ' ' + test_data['Description'][i]) for i,label in enumerate(test_data['Class Index'])]

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                  shuffle=True, collate_fn=collate_batch)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                  shuffle=False, collate_fn=collate_batch)
    
    def build_vocabulary(datasets):
        for dataset in datasets:
            for _, text in dataset:
                yield tokenizer(text)

    if vocab is None:
        # Vocabulary includes all tokens with at least 10 occurrences in the texts
        # Special tokens <PAD> and <UNK> are used for padding sequences and unknown words respectively
        vocab = build_vocab_from_iterator(build_vocabulary([train_dataset, test_dataset]), min_freq=10, specials=["<PAD>","<UNK>"])
        vocab.set_default_index(vocab["<UNK>"])

    return train_dataset, test_dataset, train_loader, test_loader, vocab

train_dataset, test_dataset, train_loader, test_loader, vocab = preprocess_data(MAX_WORDS)

target_classes = ["World", "Sports", "Business", "Sci/Tech"]


######################################################################
# Define functions to train and evaluate the model
# ------------------------------------------------

# Count model parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def EvaluateModel(model, loss_fn, val_loader):
    model.eval()
    with torch.no_grad():
        Y_actual, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_actual.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_actual = torch.cat(Y_actual)
        Y_preds = torch.cat(Y_preds)
    
    # Returns mean loss, actual labels, predicted labels 
    return torch.tensor(losses).mean(), Y_actual.detach().cpu().numpy(), Y_preds.detach().cpu().numpy()


def TrainModel(model, loss_fn, optimizer, train_loader, epochs):
    epoch_times_list = []
    for i in range(1, epochs+1):
        start = time.time()
        model.train()
        print('Epoch:',i)
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        epoch_time = (time.time() - start)
        epoch_times_list.append(epoch_time)
        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
    
    # return the average epoch duration in seconds.
    return sum(epoch_times_list) / len(epoch_times_list)


Thereafter, we alter the Model class to make it customizable. This allows us to use this class to create all alternative models of the first task, by adjusting the rnn_type, rnn_layers and bidirectional parameters, and also load pre-trained embeddings and choose to keep them freezed or not (pretrained_embeddings and freeze_embeddings parameters), for questions 4 and 5.

In [ ]:
######################################################################
# Define the model
# ----------------
class Model(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, rnn_type, rnn_layers, bidirectional, pretrained_embeddings=None, freeze_embeddings=False):
        super(Model, self).__init__()
        
        # Setting seed to ensure reproducibility
        SEED = 1
        random.seed(SEED)
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)

        if pretrained_embeddings is None:
            self.embedding_layer = nn.Embedding(num_embeddings=input_dim, embedding_dim=embedding_dim)
        else:
            # Create embedding layer with shape that matches the pretrained embeddings matrix. Setting padding index to 0, to avoid training the <PAD> embedding when embedding layer is not frezed.
            self.embedding_layer = nn.Embedding(num_embeddings=pretrained_embeddings.shape[0], embedding_dim=pretrained_embeddings.shape[1], padding_idx=0)
            self.embedding_layer.weight.data.copy_(pretrained_embeddings)
            self.embedding_layer.weight.requires_grad = not freeze_embeddings
        
        if rnn_type == "rnn":
          self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, num_layers=rnn_layers, bidirectional=bidirectional)
        elif rnn_type == "lstm":
          self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, num_layers=rnn_layers, bidirectional=bidirectional)
        else:
          raise ValueError(f"Unsupported rnn type: {rnn_type}")

        if bidirectional:
            # In the case of bidirectional RNN, its output dimension will be 2 times the hidden size, 
            # since the hidden states of the forward and the backward RNN are concatenated. 
            # Thus, we change the input dimension of the linear layer to be 2 times the hidden dimension of the RNN, to match the output of the RNN.
            self.linear = nn.Linear(2*hidden_dim, output_dim)
        else:
            self.linear = nn.Linear(hidden_dim, output_dim)

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn(embeddings)
        logits = self.linear(output[:,-1])  # The last output of RNN is used for sequence classification
        probs = F.softmax(logits, dim=1)
        return probs

In addition, we define the run_expirements method which trains and evaluates all models in an experimental_configs list and we will use it for the first question. It also supports loading of pre-trained embeddings (freezed or not), thus, we will also use it for questions 4 and 5 too.

In [ ]:
def run_experiments(experimental_configs, pretrained_embeddings=None, freeze_embeddings=False):
    results = []
    for config in experimental_configs:
        print(f"Experimental configuration: {config}")
        classifier = Model(
            input_dim=len(vocab), 
            embedding_dim=EMBEDDING_DIM, 
            hidden_dim=HIDDEN_DIM, 
            output_dim=len(target_classes),
            rnn_type=config["rnn_type"],
            rnn_layers=config["rnn_layers"],
            bidirectional=config["bidirectional"],
            pretrained_embeddings=pretrained_embeddings,
            freeze_embeddings=freeze_embeddings
        ).to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam([param for param in classifier.parameters() if param.requires_grad == True],lr=LEARNING_RATE)
        num_of_parameters = count_parameters(classifier)

        print('\nModel:')
        print(classifier)
        print('Total parameters: ', num_of_parameters)
        print('\n\n')

        average_epoch_time = TrainModel(classifier, loss_fn, optimizer, train_loader, EPOCHS)
        _, Y_actual, Y_preds = EvaluateModel(classifier, loss_fn, test_loader)

        accuracy = accuracy_score(Y_actual, Y_preds)


        print(f"Average epoch duration: {average_epoch_time:.2f} seconds.")
        print("\nTest Accuracy : {:.3f}".format(accuracy))
        print("\nClassification Report : ")
        print(classification_report(Y_actual, Y_preds, target_names=target_classes))
        print("\nConfusion Matrix : ")
        print(confusion_matrix(Y_actual, Y_preds))
        print("-" * 80 + "\n")

        results.append(
            {
                "Experiment name": config["experiment_name"],
                "Accuracy": accuracy,
                "Parameters": num_of_parameters,
                "Time cost": average_epoch_time,
                "Y_preds": Y_preds,
                "Y_actual": Y_actual
            }
        )
    return results

As a final point, we define the print_eval_table function that prints a table showcasing the results of all models, as described in the first task.

In [ ]:
from tabulate import tabulate


def print_eval_table(results):
    table = [
        [""] + [x["Experiment name"] for x in results],
        ["Accuracy"] + [x["Accuracy"] for x in results],
        ["Parameters"] + [x["Parameters"] for x in results],
        ["Time cost"] + [x["Time cost"] for x in results],
    ]
    print(tabulate(table, headers="firstrow", tablefmt="grid"))



## Answers

### 1.

Initially, we create a list of the experimental configurations that describe the 6 requested models. 

In [ ]:
experimental_configs = [
    {
        "experiment_name": "1RNN",
        "rnn_type": "rnn",
        "bidirectional": False,
        "rnn_layers": 1
    },
    {
        "experiment_name": "1Bi-RNN",
        "rnn_type": "rnn",
        "bidirectional": True,
        "rnn_layers": 1
    },
    {
        "experiment_name": "2Bi-RNN",
        "rnn_type": "rnn",
        "bidirectional": True,
        "rnn_layers": 2
    },
    {
        "experiment_name": "1LSTM",
        "rnn_type": "lstm",
        "bidirectional": False,
        "rnn_layers": 1
    },
    {
        "experiment_name": "1Bi-LSTM",
        "rnn_type": "lstm",
        "bidirectional": True,
        "rnn_layers": 1
    },
    {
        "experiment_name": "2Bi-LSTM",
        "rnn_type": "lstm",
        "bidirectional": True,
        "rnn_layers": 2
    }
]

Additionally, we create, train, and evaluate these models by using the run_experiments method giving the experimental_configs as input.

In [ ]:
results = run_experiments(experimental_configs)
print_eval_table(results)

Experimental configuration: {'experiment_name': '1RNN', 'rnn_type': 'rnn', 'bidirectional': False, 'rnn_layers': 1}

Model:
Model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): RNN(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=4, bias=True)
)
Total parameters:  2136284



Epoch: 1


100%|██████████| 118/118 [00:06<00:00, 18.53it/s]


Train Loss : 1.292
Epoch: 2


100%|██████████| 118/118 [00:04<00:00, 27.66it/s]


Train Loss : 1.051
Epoch: 3


...

Train Loss : 0.813
Epoch: 14


100%|██████████| 118/118 [00:06<00:00, 17.28it/s]


Train Loss : 0.810
Epoch: 15


100%|██████████| 118/118 [00:06<00:00, 17.50it/s]


Train Loss : 0.809
Average epoch duration: 6.87 seconds.

Test Accuracy : 0.890

Classification Report : 
              precision    recall  f1-score   support

       World       0.91      0.89      0.90      1900
      Sports       0.95      0.95      0.95      1900
    Business       0.86      0.85      0.85      1900
    Sci/Tech       0.85      0.87      0.86      1900

    accuracy                           0.89      7600
   macro avg       0.89      0.89      0.89      7600
weighted avg       0.89      0.89      0.89      7600


Confusion Matrix : 
[[1691   52   83   74]
 [  31 1799   33   37]
 [  75   25 1612  188]
 [  63   23  153 1661]]
--------------------------------------------------------------------------------

+------------+-------------+-------------+-----------+-------------+-------------+-------------+
|            |        1RNN |     1Bi-RNN |   2Bi-RNN |       1LSTM |    1Bi-LSTM |    2Bi-LSTM |
+============+=============+=============+===========+=============+=

Although the differences in Accuracy between the models is small, models with more parameters tend to perform better. 1RNN is an exception and it performs better than more complex RNNs, which could be the result of a better random initialization of this model. Nonetheless, when comparing LSTMs with RNNs, all LSTMs perform better than RNNs, even if an RNN model has more parameters. e.g. 1LSTM performs better than 2Bi-RNN even though the latter has more parameters.

Moreover, Time cost increases with the complexity (i.e. number of paramters) of the model. Models with 2 layers (vs. 1) are slower, with 2Bi-LSTM being the slowest model. Time cost was calculated on GPU runtimes and even larger differences were noticed on CPU runtimes. 

Last but not least, using 2 layers instead of 1, slightly improves the performance of both Bi-RNN and Bi-LSTM.

### 2.

In [ ]:
import numpy as np
from collections import Counter

mistakes_per_experiment = []
misstake_pairs_counter = Counter()
for r in results:
    wrong_preds_indices = np.where(r["Y_preds"] != r["Y_actual"])[0]
    mistakes_per_experiment.append(set(wrong_preds_indices.tolist()))
    for ind in wrong_preds_indices:
        actual_class = target_classes[r["Y_actual"][ind]]
        pred_class = target_classes[r["Y_preds"][ind]]
        misstake_pairs_counter.update([(actual_class, pred_class)])

common_mistakes = list(set.intersection(*mistakes_per_experiment))
print(f"There are {len(common_mistakes)} examples that were classified incorrectly by all {len(results)} experiments")

misclassfied_example = test_dataset[common_mistakes[0]]
print(f"\nExample:\n{misclassfied_example[1]}")
print(f"Actual label: {target_classes[misclassfied_example[0] - 1]}")

common_mistakes_per_class = Counter()
for mistake_ind in common_mistakes:
    label = target_classes[test_dataset[mistake_ind][0] - 1]
    common_mistakes_per_class.update([label])


print("\nNumber of common misclassifications accross models, per category:")
print(tabulate(common_mistakes_per_class.most_common()))

print("\nNumber of common misclassifications accross models, per actual class - wrong prediction pairs:")
print(tabulate(misstake_pairs_counter.most_common()))

most_common_pair = misstake_pairs_counter.most_common(1)[0]
print(f"\nThe most common actual class - wrong prediction pairs is: \nActual class: {most_common_pair[0][0]}\nPredicted class: {most_common_pair[0][1]}\nOccurrences: {most_common_pair[1]}")

There are 414 examples that were classified incorrectly by all 6 experiments

Example:
Greenspan: Debt, home prices not dangerous The record level of debt carried by American households and soaring home prices do not appear to represent serious threats to the US economy, Federal Reserve Chairman Alan Greenspan said Tuesday.
Actual label: Sci/Tech

Number of common misclassifications accross models, per category:
--------  ---
Business  155
World     122
Sci/Tech  114
Sports     23
--------  ---

Number of common misclassifications accross models, per actual class - wrong prediction pairs:
------------------------  ----
('Business', 'Sci/Tech')  1280
('Sci/Tech', 'Business')   978
('World', 'Business')      586
('Business', 'World')      513
('Sci/Tech', 'World')      482
('World', 'Sports')        435
('World', 'Sci/Tech')      420
('Sci/Tech', 'Sports')     252
('Sports', 'Sci/Tech')     212
('Business', 'Sports')     203
('Sports', 'World')        199
('Sports', 'Business')     151
-

### 3.

Next, we create new datasets, data loaders and vocab by calling preprocess_data with MAX_WORDS=50.

Consequently. we run again all 6 models by using the run_experiment function, which will now respect the new data loaders that use MAX_WORDS=50

In [ ]:
MAX_WORDS = 50
train_dataset, test_dataset, train_loader, test_loader, vocab = preprocess_data(MAX_WORDS)
results = run_experiments(experimental_configs)
print_eval_table(results)

Experimental configuration: {'experiment_name': '1RNN', 'rnn_type': 'rnn', 'bidirectional': False, 'rnn_layers': 1}

Model:
Model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): RNN(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=4, bias=True)
)
Total parameters:  2136284



Epoch: 1


100%|██████████| 118/118 [00:04<00:00, 23.87it/s]


Train Loss : 1.378
Epoch: 2


100%|██████████| 118/118 [00:04<00:00, 25.17it/s]


Train Loss : 1.336
Epoch: 3


...

Train Loss : 0.825
Epoch: 14


100%|██████████| 118/118 [00:06<00:00, 17.60it/s]


Train Loss : 0.824
Epoch: 15


100%|██████████| 118/118 [00:07<00:00, 16.00it/s]


Train Loss : 0.822
Average epoch duration: 7.14 seconds.

Test Accuracy : 0.892

Classification Report : 
              precision    recall  f1-score   support

       World       0.91      0.88      0.89      1900
      Sports       0.93      0.96      0.95      1900
    Business       0.85      0.87      0.86      1900
    Sci/Tech       0.87      0.86      0.87      1900

    accuracy                           0.89      7600
   macro avg       0.89      0.89      0.89      7600
weighted avg       0.89      0.89      0.89      7600


Confusion Matrix : 
[[1664   74  108   54]
 [  29 1824   29   18]
 [  55   22 1660  163]
 [  74   38  154 1634]]
--------------------------------------------------------------------------------

+------------+-------------+-------------+-----------+-------------+-------------+-------------+
|            |        1RNN |     1Bi-RNN |   2Bi-RNN |       1LSTM |    1Bi-LSTM |    2Bi-LSTM |
+============+=============+=============+===========+=============+=

RNN models perform significanlt worse when using MAX_WORDS=50 instead of 25. This is probably happening due to the vanishing gradiend problem of RNNs. The longer the sequence that the RNN has to encode, the more difficult it is to capture information from the first timesteps of the RNN. This problem is solved in LSTMs which manage to "remember" important information from earlier timesteps. Indeed, we notice that the LSTM models are not affected by vanishing gradient and perform similarly and slightly better than the experiments with MAX_WORDS=25.

Regarding the complexity of the models, it remains the same regardless of the MAX_WORDS values, since its value only affects the number of timesteps/calculations and not the structure of the neural network.

### 4.

Now, we use torchtext to load the GloVe embeddings. First, we create a vocabulary for this experiment, by finding the common words between the vocab that we costructed previously from the dataset, and the vocabulary of the GloVe embeddings. This way, we will only load the embeddings that we need for the training and evaluation of the model. Using these common words, we create a new torchtext vocab that also contains \<PAD> and \<UNK> tokens at the first two positions. We then create the embedding matrix by taking the embedding of each word in the new vocab and create a matrix whose rows are these embeddings in the same order as in the vocabulary. We set the first row to zeros (for the <PAD> token) and the second vector to a random vector (for the <UNK> token). 

To load the embeddings matrix, we modified the model so that:
 - The embedding layer is created with the same dimensions as the pre-trained mebeddings matrix to load.
 - We explicitly set padding_idx=0 to prevent the \<PAD> embedding from updating during training.
 - We replace the randomly initialized embedding layer weight, with the pre-trained embeddings.

self.embedding_layer = nn.Embedding(num_embeddings=pretrained_embeddings.shape[0], embedding_dim=pretrained_embeddings.shape[1], padding_idx=0)

self.embedding_layer.weight.data.copy_(pretrained_embeddings)

In [ ]:
import torchtext
from collections import OrderedDict
pretrained_embeddings = torchtext.vocab.GloVe(name='6B', dim=100)

# Creating the new embeddings vocabulary, by keeping common terms between our original dataset vocabualry, and the Glove-6B-100D vocabulary. 
embeddings_vocab = set(pretrained_embeddings.itos)
embeddings_vocab = [t for t in vocab.get_itos() if t in embeddings_vocab]

# Creating an OrderedDict with dummy token frequencies (1), in order to build a torchtext vocab.
vocab = OrderedDict([(token, 1) for token in embeddings_vocab])

# Building torchtext vocab, with <PAD> and <UNK> special tokens at the first positions (0 and 1 respectively).
vocab = torchtext.vocab.vocab(vocab, min_freq=1, specials=["<PAD>","<UNK>"], special_first=True)
vocab.set_default_index(vocab["<UNK>"])

embeddings_matrix = pretrained_embeddings.get_vecs_by_tokens(vocab.get_itos())
# Add add zeros-vector (for <PAD> token) and random vector (for <UNK> token) at the first two rows of the embeddings matrix.
embeddings_matrix[0, :] = torch.zeros(1, embeddings_matrix.shape[1]) # Setting the 0th vector to zeros (for the <PAD> token)
embeddings_matrix[1, :] = torch.rand(1, embeddings_matrix.shape[1]) # Setting the 1st vector to random vector (for the <UNK> token)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.36MB/s]                           
100%|█████████▉| 399999/400000 [00:18<00:00, 22058.35it/s]


In [ ]:
MAX_WORDS = 25 # Setting MAX_WORDS back to 25 for Γ4.
train_dataset, test_dataset, train_loader, test_loader, vocab = preprocess_data(MAX_WORDS, vocab)
results = run_experiments(experimental_configs, pretrained_embeddings=embeddings_matrix, freeze_embeddings=False)
print_eval_table(results)

Experimental configuration: {'experiment_name': '1RNN', 'rnn_type': 'rnn', 'bidirectional': False, 'rnn_layers': 1}

Model:
Model(
  (embedding_layer): Embedding(20437, 100, padding_idx=0)
  (rnn): RNN(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=4, bias=True)
)
Total parameters:  2054584



Epoch: 1


100%|██████████| 118/118 [00:04<00:00, 28.02it/s]


Train Loss : 1.053
Epoch: 2


100%|██████████| 118/118 [00:05<00:00, 23.29it/s]


Train Loss : 0.901
Epoch: 3


...

Train Loss : 0.804
Epoch: 14


100%|██████████| 118/118 [00:06<00:00, 18.46it/s]


Train Loss : 0.803
Epoch: 15


100%|██████████| 118/118 [00:07<00:00, 16.54it/s]


Train Loss : 0.801
Average epoch duration: 6.71 seconds.

Test Accuracy : 0.907

Classification Report : 
              precision    recall  f1-score   support

       World       0.93      0.90      0.91      1900
      Sports       0.96      0.97      0.96      1900
    Business       0.88      0.86      0.87      1900
    Sci/Tech       0.86      0.90      0.88      1900

    accuracy                           0.91      7600
   macro avg       0.91      0.91      0.91      7600
weighted avg       0.91      0.91      0.91      7600


Confusion Matrix : 
[[1701   57   75   67]
 [  19 1840   16   25]
 [  66   15 1640  179]
 [  45   10  130 1715]]
--------------------------------------------------------------------------------

+------------+-------------+-------------+------------+-------------+-------------+-------------+
|            |        1RNN |     1Bi-RNN |    2Bi-RNN |       1LSTM |    1Bi-LSTM |    2Bi-LSTM |
+============+=============+=============+============+============

We observe that all models perform better when using pre-trained embeddings vs. random embedding initialization. The best model with pre-trained embeddings is 1Bi-LSTM with accuracy: 0.908158. The best model with random embeddings initialization (task 1) was 2Bi-LSTM with accuracy: 0.889868. That is, the usage of pre-trained embeddings lead to singificant improvements. The complexity of the models is similar to that of the models in first task, since we just changed the way we initialize the weights of the embeddings, which are still parameters to be optimized during training. The number of parameters in the models with pre-trained embeddings is slightly smaller that in the models of task 1 only due to the slightly smaller vocabulary size, since not all words of our original vocabualry had a cooresponding GloVe embedding.

### 5.

In this task, the pre-trained embeddings should be modified during model training (freeze embeddings). To achieve this, we modified the code so that gradient calculation is not required for the embedding layer, as follows:

self.embedding_layer.weight.requires_grad = not freeze_embeddings

i.e. when we want to freeze the embeddings this translates to:

self.embedding_layer.weight.requires_grad = False

During the creation of the optimizer, this layer will be ingored because its requires_grad variable is False, and the layer will not be optimized during training:

optimizer = torch.optim.Adam([param for param in classifier.parameters() if param.requires_grad == True],lr=LEARNING_RATE)


In [ ]:
MAX_WORDS = 25
train_dataset, test_dataset, train_loader, test_loader, vocab = preprocess_data(MAX_WORDS, vocab)
results = run_experiments(experimental_configs, pretrained_embeddings=embeddings_matrix, freeze_embeddings=True)
print_eval_table(results)

Experimental configuration: {'experiment_name': '1RNN', 'rnn_type': 'rnn', 'bidirectional': False, 'rnn_layers': 1}

Model:
Model(
  (embedding_layer): Embedding(20437, 100, padding_idx=0)
  (rnn): RNN(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=4, bias=True)
)
Total parameters:  10884



Epoch: 1


100%|██████████| 118/118 [00:04<00:00, 27.48it/s]


Train Loss : 1.063
Epoch: 2


100%|██████████| 118/118 [00:05<00:00, 23.00it/s]


Train Loss : 0.921
Epoch: 3


...

Train Loss : 0.836
Epoch: 14


100%|██████████| 118/118 [00:05<00:00, 22.78it/s]


Train Loss : 0.834
Epoch: 15


100%|██████████| 118/118 [00:05<00:00, 19.80it/s]


Train Loss : 0.832
Average epoch duration: 5.50 seconds.

Test Accuracy : 0.900

Classification Report : 
              precision    recall  f1-score   support

       World       0.93      0.89      0.91      1900
      Sports       0.94      0.98      0.96      1900
    Business       0.87      0.84      0.86      1900
    Sci/Tech       0.86      0.89      0.87      1900

    accuracy                           0.90      7600
   macro avg       0.90      0.90      0.90      7600
weighted avg       0.90      0.90      0.90      7600


Confusion Matrix : 
[[1687   64   87   62]
 [  15 1853   18   14]
 [  62   27 1603  208]
 [  54   17  133 1696]]
--------------------------------------------------------------------------------

+------------+--------------+--------------+--------------+--------------+--------------+---------------+
|            |         1RNN |      1Bi-RNN |      2Bi-RNN |        1LSTM |     1Bi-LSTM |      2Bi-LSTM |
+============+==============+==============+=======

With freezed pre-trained embeddings, the models perform slightly worse than with non freezed pre-trained embeddings. Despite this, we notice that the complexity of the models is much smaller, since the parameters of the models are now only the RNN/LSTM parameters and the linear layer parameters. In the previous models, the majority of the parameters originated from the embedding layer, which is now freezed during training.

### 6.

To complete the last task of this assignment, we modify the preprocessing process to respect the new dataset format, which contains a "review" column for the text to classify, and a "sentiment" column for the label ("positive" or "negative"). 

In [ ]:
import pandas as pd
data = pd.read_csv('IMDB Dataset.csv')

In [ ]:
import math
import random

def preprocess_imdb_data(max_words):

    # All texts are truncated and padded to max_words tokens
    def collate_batch(batch):
        Y, X = list(zip(*batch))
        Y = [0 if label == "negative" else 1 for label in Y]
        Y = torch.tensor(Y, dtype=torch.float)
        X = [vocab(tokenizer(text)) for text in X]
        # Bringing all samples to max_words length. Shorter texts are padded with <PAD> sequences, longer texts are truncated.
        X = [tokens+([vocab['<PAD>']]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X]
        return torch.tensor(X, dtype=torch.int32).to(device), Y.to(device) 

    dataset = [(row["sentiment"], row["review"]) for i, row in data.iterrows()]

    random.seed(1)
    random.shuffle(dataset)

    train_dataset = dataset[:math.floor(0.8*len(dataset))]
    test_dataset = dataset[math.floor(0.8*len(dataset)):]

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                  shuffle=True, collate_fn=collate_batch)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                  shuffle=False, collate_fn=collate_batch)
    
    def build_vocabulary(datasets):
        for dataset in datasets:
            for _, text in dataset:
                yield tokenizer(text)

    # Vocabulary includes all tokens with at least 10 occurrences in the texts
    # Special tokens <PAD> and <UNK> are used for padding sequences and unknown words respectively
    vocab = build_vocab_from_iterator(build_vocabulary([train_dataset, test_dataset]), min_freq=10, specials=["<PAD>","<UNK>"])
    vocab.set_default_index(vocab["<UNK>"])

    return train_dataset, test_dataset, train_loader, test_loader, vocab

train_dataset, test_dataset, train_loader, test_loader, vocab = preprocess_imdb_data(MAX_WORDS)

target_classes = ["negative", "positive"]


Furthermore, we modify the model into a binary classification model, which has an output dimension of 1 and uses a sigmoid activation function instead of softmax after the linear layer. The model will predict a number between 0 and 1 and the predictions closer to 0 will be interpreted as "negative" and closer to 1 as "positive". 

In [ ]:
# Creating a Binary Classification model that always has output dimension of 1 and uses sigmoid instead of softmax.

class BinaryClassificationModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, rnn_type, rnn_layers, bidirectional):
        super(BinaryClassificationModel, self).__init__()

        self.embedding_layer = nn.Embedding(num_embeddings=input_dim, embedding_dim=embedding_dim)
        
        if rnn_type == "rnn":
          self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, num_layers=rnn_layers, bidirectional=bidirectional)
        elif rnn_type == "lstm":
          self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, num_layers=rnn_layers, bidirectional=bidirectional)
        else:
          raise ValueError(f"Unsupported rnn type: {rnn_type}")

        if bidirectional:
            # In the case of bidirectional RNN, its output dimension will be 2 times the hidden size, 
            # since the hidden states of the forward and the backward RNN are concatenated. 
            # Thus, we change the input dimension of the linear layer to be 2 times the hidden dimension of the RNN, to match the output of the RNN.
            self.linear = nn.Linear(2*hidden_dim, 1)
        else:
            self.linear = nn.Linear(hidden_dim, 1)

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn(embeddings)
        logits = self.linear(output[:,-1])  # The last output of RNN is used for sequence classification
        probs = F.sigmoid(logits)
        return probs.squeeze(-1)

We also change the loss to Binary Cross Entropy loss (BCELoss), to comply with the new model output.

In [ ]:
def run_imdb_experiments(experimental_configs, pretrained_embeddings=None, freeze_embeddings=False):
    results = []
    for config in experimental_configs:
        print(f"Experimental configuration: {config}")
        classifier = BinaryClassificationModel(
            input_dim=len(vocab), 
            embedding_dim=EMBEDDING_DIM, 
            hidden_dim=HIDDEN_DIM, 
            rnn_type=config["rnn_type"],
            rnn_layers=config["rnn_layers"],
            bidirectional=config["bidirectional"]
        ).to(device)
        # Using Binary CrossEntropy loss, since the Binary Classification model will return a sinlge probability in the range (0, 1).
        loss_fn = nn.BCELoss()
        optimizer = torch.optim.Adam([param for param in classifier.parameters() if param.requires_grad == True],lr=LEARNING_RATE)
        num_of_parameters = count_parameters(classifier)

        print('\nModel:')
        print(classifier)
        print('Total parameters: ', num_of_parameters)
        print('\n\n')

        average_epoch_time = TrainIMDBModel(classifier, loss_fn, optimizer, train_loader, EPOCHS)
        _, Y_actual, Y_preds = EvaluateIMDBModel(classifier, loss_fn, test_loader)
        accuracy = accuracy_score(Y_actual, Y_preds)


        print(f"Average epoch duration: {average_epoch_time:.2f} seconds.")
        print("\nTest Accuracy : {:.3f}".format(accuracy))
        print("\nClassification Report : ")
        print(classification_report(Y_actual, Y_preds, target_names=target_classes))
        print("\nConfusion Matrix : ")
        print(confusion_matrix(Y_actual, Y_preds))
        print("-" * 80 + "\n")

        results.append(
            {
                "Experiment name": config["experiment_name"],
                "Accuracy": accuracy,
                "Parameters": num_of_parameters,
                "Time cost": average_epoch_time,
                "Y_preds": Y_preds,
                "Y_actual": Y_actual
            }
        )
    return results

In [ ]:
def EvaluateIMDBModel(model, loss_fn, val_loader):
    model.eval()
    with torch.no_grad():
        Y_actual, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_actual.append(Y)
            Y_preds.append(torch.tensor([0 if p <= 0.5 else 1 for p in preds]))
        Y_actual = torch.cat(Y_actual)
        Y_preds = torch.cat(Y_preds)
    
    # Returns mean loss, actual labels, predicted labels 
    return torch.tensor(losses).mean(), Y_actual.detach().cpu().numpy(), Y_preds.detach().cpu().numpy()


def TrainIMDBModel(model, loss_fn, optimizer, train_loader, epochs):
    epoch_times_list = []
    for i in range(1, epochs+1):
        start = time.time()
        model.train()
        print('Epoch:',i)
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        epoch_time = (time.time() - start)
        epoch_times_list.append(epoch_time)
        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
    
    # return the average epoch duration in seconds.
    return sum(epoch_times_list) / len(epoch_times_list)

In [ ]:
MAX_WORDS = 25
train_dataset, test_dataset, train_loader, test_loader, vocab = preprocess_imdb_data(MAX_WORDS)
results = run_imdb_experiments(experimental_configs)
print_eval_table(results)

Experimental configuration: {'experiment_name': '1RNN', 'rnn_type': 'rnn', 'bidirectional': False, 'rnn_layers': 1}

Model:
BinaryClassificationModel(
  (embedding_layer): Embedding(29065, 100)
  (rnn): RNN(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=1, bias=True)
)
Total parameters:  2917189



Epoch: 1


100%|██████████| 40/40 [00:06<00:00,  6.06it/s]


Train Loss : 0.696
Epoch: 2


100%|██████████| 40/40 [00:07<00:00,  5.65it/s]


Train Loss : 0.689
Epoch: 3


...

Train Loss : 0.226
Epoch: 14


100%|██████████| 40/40 [00:07<00:00,  5.21it/s]


Train Loss : 0.185
Epoch: 15


100%|██████████| 40/40 [00:06<00:00,  5.83it/s]


Train Loss : 0.155
Average epoch duration: 7.41 seconds.

Test Accuracy : 0.721

Classification Report : 
              precision    recall  f1-score   support

    negative       0.72      0.74      0.73      5104
    positive       0.72      0.70      0.71      4896

    accuracy                           0.72     10000
   macro avg       0.72      0.72      0.72     10000
weighted avg       0.72      0.72      0.72     10000


Confusion Matrix : 
[[3768 1336]
 [1457 3439]]
--------------------------------------------------------------------------------

+------------+-------------+-------------+-------------+-------------+-------------+-------------+
|            |        1RNN |     1Bi-RNN |     2Bi-RNN |       1LSTM |    1Bi-LSTM |    2Bi-LSTM |
+============+=============+=============+=============+=============+=============+=============+
| Accuracy   | 0.7137      | 0.7155      | 0.7102      | 0.7131      | 0.717       | 0.7207      |
+------------+-------------+-------------

In this new dataset, again more complex models tend to perform slightly better. An exception is the 2Bi-RNN model which performs worse than less complex RNNs. The most complex model (2Bi-LSTM) achieves the best accuracy.

Regarding Time cost, the more complex a model, the slower is its average epoch runtime, although the differences in GPU execution is small. The slowest model is again the most complex model, 2Bi-LSTM.

Using 2 layers instead of 1, results in lower accuracy in the Bi-RNN models, but improves accuracy of the Bi-LSTM models and leads to the best model: 2Bi-LSTM.